In [1]:
from simulation_utils import * 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import gpboost as gpb
import pdpbox
from pdpbox import pdp
sns.set()

# Full data setting

## Non-linear function; Shared GP in data generating process

In [2]:
# Generate data
n, m = 2000, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20 
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', random_state=60, shared_gp=True)

# Create groups
groups = np.arange(n)
for i in range(m):
    groups[i*p:(i+1)*p] = i

# Create times for validation datasets
times = np.tile(np.arange(p)*10+10, m)

## GPBoost hyperparameter tuning

### Random Intercept model

In [3]:
# Random slope
gp_model = gpb.GPModel(group_data=np.column_stack((groups, times)))
# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=0,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 



 |█-------------------------------------------------| 1.7857% 



 |█-------------------------------------------------| 2.6786% 



 |██------------------------------------------------| 3.5714% 



 |██------------------------------------------------| 4.4643% 



 |███-----------------------------------------------| 5.3571% 



 |███-----------------------------------------------| 6.2500% 



 |████----------------------------------------------| 7.1429% 



 |████----------------------------------------------| 8.0357% 



 |████----------------------------------------------| 8.9286% 



 |█████---------------------------------------------| 9.8214% 



 |█████---------------------------------------------| 10.7143% 



 |██████--------------------------------------------| 11.6071% 



 |██████--------------------------------------------| 12.5000% 



 

### GPBoost with independent GPs

In [4]:
# Independent GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cluster_ids=groups, cov_function='matern', cov_fct_shape=2.5)

# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 1 of 112: {'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.5557845985829477) found for the following parameter combination:
{'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 999.0}


 |█-------------------------------------------------| 1.7857% 

Trying parameter combination 2 of 112: {'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.5360086565328326) found for the following parameter combination:
{'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 269.8}


 |█-------------------------------------------------| 2.6786% 

Trying parameter combination 3 of 112: {'learning_rate': 0.5, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |██------------------------------------------------| 3.571

KeyboardInterrupt: 

### GPBoost with shared GPs

In [3]:
# Shared GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cov_function='matern', cov_fct_shape=2.5)

# Use the same hyperparameters as for Independent GP and only cross-validate to find num_boost_round
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 1,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid={},
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 1 parameter combinations...


 |██████████████████████████████████████████████████| 100.0000% 


Trying parameter combination 1 of 1: {} ...
***** New best score (1.52064624732644) found for the following parameter combination:
{'num_boost_round': 58.4}
Best number of iterations:  58.4
Best score:  1.52064624732644
Best parameters:  {}


### Gradient-boosted trees including groups as a categorical variable

In [ ]:
# GBT with grouping variable as a categorical feature
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', shared_gp=True, random_state=60, include_cat_feature=True)

# Candidate parameter grid
param_grid = {'num_leaves': [10, 20, 50, 200, 500, 1000, 5000, 2**10], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 5, 10, 20, 30, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'learning_rate' : 0.10}
          #'device_type' : 'gpu'}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=None,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets_cat['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 336 parameter combinations...


 |--------------------------------------------------| 0.2976% 

Trying parameter combination 1 of 336: {'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.593675135547522) found for the following parameter combination:
{'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 155.6}


 |--------------------------------------------------| 0.5952% 

Trying parameter combination 2 of 336: {'num_leaves': 20, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 3 of 336: {'num_leaves': 50, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.1905% 

Trying parameter combination 4 of 336: {'num_leaves': 200, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.4881% 

Trying parameter combin

In [3]:
# Create results dataframe
results_1 = pd.DataFrame(columns=["RMSE Interpolation (mean)", "RMSE Interpolation (std)", "RMSE Extrapolation (mean)", "RMSE Extrapolation (std)", "RMSE_F (mean)", "RMSE_F (std)", "Time"],
                         index=["Linear Mixed Effects Model with no fixed features (random intercept)",
                                "Linear Mixed Effects Model with Random Intercept",
                                "Linear Mixed Effects Model with Shared Gaussian Process (Matern-5/2)",
                                "Linear Mixed Effects Model with Inividual Gaussian Process (Matern-5/2)",
                                'Gradient-boosted tree with group as categorical variable (no random effects)',
                                "GPBoost with Random Intercept",
                                "GPBoost with Shared Gaussian Process (Matern-5/2)",
                                "GPBoost with Independent Gaussian Process (Matern-5/2)"])

In [4]:
# 1. Linear Mixed Effects Model with no fixed features (random intercept)
time_list, RMSE_list1, RMSE_list2 = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False, no_features=True)

results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (std)"] = np.std(RMSE_list2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
# 2. Linear Mixed Effects Model with Random Intercept
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False)

results_1.loc["Linear Mixed Effects Model with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# 3. Linear Mixed Effects Model with Shared Gaussian Process and Matern-5/2 kernel
mod_name = 'Linear Mixed Effects Model with Shared Gaussian Process (Matern-5/2)'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=True, kernel='matern', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
# 4. Linear Mixed Effects Model with Individual Gaussian Process and Matern-5/2 kernel
mod_name = 'Linear Mixed Effects Model with Individual Gaussian Process (Matern-5/2)'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=False, kernel='matern', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

Error encountered
Error encountered


In [8]:
# 5. Gradient tree-boosting including the grouping variable as a categorical variable ('Boosting_Cat')
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', shared_gp=True, random_state=60, include_cat_feature=True)

params = {'num_leaves': 1024, 'max_depth': 2, 'min_data_in_leaf': 1}
time_list, RMSE_list1, RMSE_list2, _ = train_and_test(datasets_cat, n_datasets, merf=False, GPBoost_cat=True, linear=False, GP=False, num_boost_round=101, params=params)

results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "Time"] = np.mean(time_list)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (std)"] = np.std(RMSE_list2) 

  0%|          | 0/20 [00:00<?, ?it/s]

[GPBoost] [Info] Total Bins 1315
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 6
[GPBoost] [Info] Start training from score 4.049996
Error encountered
[GPBoost] [Info] Total Bins 1313
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 6
[GPBoost] [Info] Start training from score 3.836820
Error encountered
[GPBoost] [Info] Total Bins 1314
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 6
[GPBoost] [Info] Start training from score 3.880845
Error encountered
[GPBoost] [Info] Total Bins 1314
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 6
[GPBoost] [Info] Start training from score 4.021637
Error encountered
[GPBoost] [Info] Total Bins 1314
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 6
[GPBoost] [Info] Start training from score 3.753667
Error encountered
[GPBoost] [Info] Total Bins 1313
[G

In [9]:
# 6. GPBoost with Random Intercept
params = {'learning_rate': 0.1, 'max_depth': 3, 'min_data_in_leaf': 20}
          
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=71, params=params, merf=False, linear=False, GP=False)
results_1.loc["GPBoost with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

[GPBoost] [Info] Total Bins 1020
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 4
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=4.049996
[GPBoost] [Info] Start training from score 4.049996
[GPBoost] [Info] Total Bins 1020
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 4
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=3.836820
[GPBoost] [Info] Start training from score 3.836820
[GPBoost] [Info] Total Bins 1020
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 4
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=3.880845
[GPBoost] [Info] Start training from score 3.880845
[GPBoost] [Info] Total Bins 1020
[GPBoost] [Info] Number of data points in the train set: 1200, number of used features: 4
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=4.021637
[GPBoost] [Info] Start training from score 4.021637
[GPBoost] [I

In [10]:
# 7. GPBoost with Shared Gaussian Process
params = params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 1,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=58, params=params, merf=False, linear=False, GP=True, shared=True, kernel='matern', matern_param=2.5)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Shared Gaussian Process (Matern-5/2)", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
# 8. GPBoost with Independent Gaussian Process
params = params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 1,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=59, params=params, merf=False, linear=False, GP=True, shared=False, kernel='matern', matern_param=2.5)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Independent Gaussian Process (Matern-5/2)", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

Error encountered
Error encountered


In [22]:
results_1

,RMSE Interpolation (mean),RMSE Interpolation (std),RMSE Extrapolation (mean),RMSE Extrapolation (std),RMSE_F (mean),RMSE_F (std),Time
Linear Mixed Effects Model with no fixed features (random intercept),1.591384,0.05997,1.815867,0.139977,NaN,NaN,0.023017
Linear Mixed Effects Model with Random Intercept,1.371958,0.052666,1.660798,0.146279,1.266863,0.055007,0.019403
Linear Mixed Effects Model with Shared Gaussian Process (Matern-5/2),1.372898,0.052541,1.660597,0.147023,1.267183,0.055142,41.816307
Linear Mixed Effects Model with Individual Gaussian Process (Matern-5/2),1.372682,0.055601,1.66206,0.153846,1.26264,0.049385,0.057646
Gradient-boosted tree with group as categorical variable (no random effects),1.341931,0.101069,1.586089,0.16501,NaN,NaN,0.029488
GPBoost with Random Intercept,1.2285,0.059714,1.5724,0.165711,0.329575,0.058003,0.532364
GPBoost with Shared Gaussian Process (Matern-5/2),1.227644,0.057195,1.572706,0.166827,0.33309,0.05778,99.752183
GPBoost with Independent Gaussian Process (Matern-5/2),1.223181,0.058782,1.577496,0.173657,0.319947,0.037375,0.735852


In [23]:
results_1.to_csv('full_data_sharedGP_friedman.csv')